In [6]:
import requests
import pandas
import numpy
from bs4 import BeautifulSoup

# Scraping Data

#### Getting Team Abrviations

In [67]:
# pulling table of team names and abreviatiosn from wikipedia
url = 'https://en.wikipedia.org/wiki/Wikipedia:WikiProject_National_Basketball_Association/National_Basketball_Association_team_abbreviations'
r = requests.get(url)
soup = BeautifulSoup(r.content)

In [17]:
# extracting just the table
table = soup.find('table',{'class':'wikitable sortable'})

In [62]:
# iterating through the table getting each row
team = []
for row in table.find_all('tr'):
    column = row.find_all('td')
    to_append = [x.get_text().strip() for x in column]
    team.append(to_append)

In [68]:
# splitting into two lists
# team_abrev = all teams 3 letter abreviations
# team_names = all teams full names
team_abrev = [x[0] for x in team[1:]]
team_names = [x[1] for x in team[1:]]

In [97]:
team_abrev

['ATL',
 'BKN',
 'BOS',
 'CHA',
 'CHI',
 'CLE',
 'DAL',
 'DEN',
 'DET',
 'GSW',
 'HOU',
 'IND',
 'LAC',
 'LAL',
 'MEM',
 'MIA',
 'MIL',
 'MIN',
 'NOP',
 'NYK',
 'OKC',
 'ORL',
 'PHI',
 'PHX',
 'POR',
 'SAC',
 'SAS',
 'TOR',
 'UTA',
 'WAS']

#### Getting the Data for 1 team for 1 season

In [73]:
team_test = 'WAS'
year_test = '2000'
url_test = 'https://www.basketball-reference.com/teams/{}/{}_games.html'.format(team_test,year_test)

In [75]:
r = requests.get(url_test)
soup = BeautifulSoup(r.content)

In [83]:
table = soup.find('table',{'class':'sortable stats_table'})
table_body = table.find('tbody')

In [94]:
table_info = []
for row in table_body.find_all('tr'):
    column = row.find_all('td')
    row_value = [x.get_text() for x in column]
    #print(row_value)
    table_info.append(row_value)

In [96]:
table_info[0]

['Tue, Nov 2, 1999',
 '',
 'Box Score',
 '',
 'Atlanta Hawks',
 'W',
 '',
 '94',
 '87',
 '1',
 '0',
 'W 1',
 '']

`table_info` as of now is just the list of games with the most basic of data. It **does not** include the stats of each game. It includes:
* date
* BLANK
* LINK TO BOX SCORE
* @ if away, BLANK if home
* opponent
* win or loss
* BLANK
* points scored
* opponent points scored
* wins that season
* losses that season
* streak of win or losses
* notes (always blank)

### Getting baseline data for 1 season

In [ ]:
table_info = []
year = 2000
for team in team_abrev:
    url = 'https://www.basketball-reference.com/teams/{}/{}_games.html'.format(team,year)
    r = requests.get(url_test)
    soup = BeautifulSoup(r.content)
    table = soup.find('table',{'class':'sortable stats_table'})
    table_body = table.find('tbody')
    table_info = []
    for row in table_body.find_all('tr'):
        column = row.find_all('td')
        row_value = [x.get_text() for x in column]
        table_info.append(row_value)

#### Teams to Have Changed their name since 2000

* 2001 - Vanvouver Grizzlies --> Memphis Grizzlies
* 2002 - Charlotte Hornets --> New Orleans Hornets
* 2005 - New Orleans Hornets --> New Orleans/Oklahoma City Hornets
* 2007 - New Orleans/Oklahoma City Hornets --> New Orleans Hornets
* 2008 - Seattle SuperSonics --> Oklahoma City Thunder
* 2012 - New Jersey Nets --> Brooklyn Nets
* 2013 - New Orleans Hornets --> New Orleans Pelicans
* 2014 - Charlotte Bobcats --> Charlotte Hornets/